<a href="https://colab.research.google.com/github/gabrielbribeiroo/OperationalResearch_UFPB/blob/main/Problema_Cobertura_Maxima.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problema da Cobertura Máxima
## Descrição do problema

Imagine que uma prefeitura tem 4 opções de terrenos e queira construir 2 escolas novas, para atender uma população de crianças localizadas em bairros em diferentes pontos da cidade. Consideramos que uma escola pode atender crianças que residem a uma distância de até 3 km da escola.

Dadas as opções de terrenos (representados pelos quadrados no diagrama) e a distribuição da população das crianças nos bairros (representados pelos círculos), quais locais vão garantir que a maior quantidade de crianças sejam atendidas?

MC1(1).drawio.svg

## Modelagem do problema

#### Variáveis:
* $x_{i} \in \{0, 1\}$ : Se o terreno i será selecionado
* $y_{j} \in \{0, 1\}$ : Se o bairro j será atendido por pelo menos uma escola

#### Função Objetivo:
* Maximizar os bairros atendidos

#### Restrições:
* Não podemos abrir mais que duas escola
* Cada bairro só será atendido se pelo menos um dos locais que o cobrem for selecionado

### A modelagem do exemplo acima seria:
$$\max y_{1} + y_{2} + y_{3} + y_{4} + y_{5} + y_{6} + y_{7} + y_{8} + y_{9} + y_{10}$$

$S.a.:$
$$x_{1} + x_{2} + x_{3} + x_{4} = 2$$
$$y_{1} ≤ x_{3}$$
$$y_{2} ≤ x_{2}$$
$$y_{3} ≤ x_{1} + x_{4}$$
$$y_{4} ≤ x_{4}$$
$$y_{5} ≤ x_{2}$$
$$y_{6} ≤ x_{4}$$
$$y_{7} ≤ x_{2}$$
$$y_{8} ≤ x_{2}$$
$$y_{9} ≤ x_{3}$$
$$y_{10} ≤ x_{1} + x_{3}$$
$$x_{1}, x_{2}, x_{3}, x_{4}, y_{1}, y_{2}, y_{3}, y_{4}, y_{5}, y_{6}, y_{7}, y_{8}, y_{9}, y_{10} \in \{0, 1\}$$

## Solucionando o problema usando Python MIP

In [ ]:
!pip install mip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 53.8 MB/s eta 0:00:00


In [ ]:
from mip import *

In [ ]:
# Instanciando o modelo
model_MCP = Model(sense=MAXIMIZE, solver_name=CBC)
model_MCP.verbose = 0

# Adicionando as variáveis dos locais a seres escolhidos e os bairros a serem cobertos
x = [model_MCP.add_var(var_type=BINARY, name = f"x_{i+1}") for i in range(4)]
y = [model_MCP.add_var(var_type=BINARY, name = f"y_{j+1}") for j in range(10)]

# Adicionando a função objetivo
model_MCP.objective = xsum(y[i] for i in range(10))

#  Adicionando as restrições
#  lembrando que, já que o índice das listas começam em 0, c1 vira c[0], l1 vira l[0], c2 vira c[1] e assim por diante
model_MCP += x[0]+x[1]+x[2]+x[3] == 2
model_MCP += y[0] <= x[2]
model_MCP += y[1] <= x[1]
model_MCP += y[2] <= x[0]+x[3]
model_MCP += y[3] <= x[3]
model_MCP += y[4] <= x[1]
model_MCP += y[5] <= x[3]
model_MCP += y[6] <= x[1]
model_MCP += y[7] <= x[1]
model_MCP += y[8] <= x[2]
model_MCP += y[9] <= x[0]+x[2]

# Resolvendo o modelo
model_MCP.optimize()

print("Valor da solução = ", model_MCP.objective_value)

print("\nLocais escolhidos:")
for v in model_MCP.vars:
    if v.name[0] == "x" and v.x > 0.00001:
        print(v.name)
print("\nBairros cobertos:")
for v in model_MCP.vars:
    if v.name[0] == "y" and v.x > 0.00001:
        print(v.name)

Valor da solução =  7.0

Locais escolhidos:
x_2
x_3

Bairros cobertos:
y_1
y_2
y_5
y_7
y_8
y_9
y_10


## Generalizando o problema

No Problema da Cobertura Máxima, precisamos escolher, entre um conjunto de locais candidatos, um subconjunto de tamanho determinado para estabelecer facilidades que atendam a maior quantidade de clientes, dado o raio de cobertura da facilidade em questão.

No exemplo dado, os locais candidatos eram os terrenos, as facilidades eram escolas e os clientes crianças. Porém, podemos tratar de diversos outros tipos de facilidades, como hospitais, delegacias, supermercados, antenas etc, e os clientes serão o grupo de interesse a quem se deseja atender.

Uma observação importante é que diferentes clientes podem ter diferentes demandas. No exemplo dado, consideramos que os bairros têm população igual. Porém, podemos ter bairros mais populosos que outros. Para isso, podemos atribuir coeficientes diferentes (pesos) às variáveis na função objetivo, para levar isso em conta.

Assim, uma forma geral de representar o PCM pode ser:

#### Entrada:
* $L$: conjunto de locais candidatos
* $C$: conjunto de clientes
* $p$: número de facilidades a serem abertas
* $h_j$: demanda do cliente $j \in C$
* $d_{ij}$: distância entre o local $i \in L$ e o cliente $j \in C$
* $R$: raio de cobertura das facilidades

#### Variáveis:
* $x_{i} \in \{0, 1\}$ : Se o local $i$ será selecionado
* $y_{j} \in \{0, 1\}$ : Se o cliente $j$ será atendido por pelo menos uma facilidade

#### Função Objetivo:
* Maximizar a soma das demandas dos clientes atendidos

#### Restrições:
* Devemos selecionar apenas $p$ locais candidatos
* Cada cliente só será atendido se pelo menos um dos locais que o cobrem for selecionado

#### Modelagem:
$$\max \sum_{j \in C} h_{j}*y_{j} $$

$S.a.:$
$$ \sum_{i \in L} x_{i} = p$$
$$ y_{j} ≤ \sum_{i \in L: d_{ij} \leq R} x_{i}, \space \forall j \in C$$
$$x_{i} \in \{0, 1\}, \space \forall i \in L$$
$$y_{j} \in \{0, 1\}, \space \forall j \in C$$

In [ ]:
# Usando a generalização, vamos criar uma função que instancia um PCM

def create_MCP_model(n_loc: int, n_fac: int, n_cli: int, rad: float, demands: List[float], distances: List[List[float]]):
    """
    Parameters:
    -----------
    n_loc: int
        Número de locais candidatos
    n_fac: int
        Número de facilidades a serem construídas, ou o número de locais a serem escolhidos
    n_cli: int
        Número de clientes
    rad: float
        Raio de cobertura das facilidades
    demands: List[int]]
        Lista com a demanda de cada cliente
        demand[j-1] deve representar hj, ou seja, a demanda do cliente j
    distances: List[List[int]]
        Matriz de tamanho n_loc x n_cli, com as distâncias entre cada cliente e cada local candidato
        distances[i-1][j-1] deve representar dij, ou seja, a distância entre o local i e o cliente j

    Returns:
    --------
    model: mip.Model
        Modelo já instanciado e pronto para ser otimizado
    """

    # Instanciando o modelo
    model = Model(sense=MAXIMIZE, solver_name=CBC)
    model.verbose = 0

    # Adicionando as variáveis dos locais a seres escolhidos e os clientes a serem cobertos
    x = [model.add_var(var_type=BINARY, name = f"x_{i+1}") for i in range(n_loc)]
    y = [model.add_var(var_type=BINARY, name = f"y_{j+1}") for j in range(n_cli)]

    # Adicionando a função objetivo
        # que é o somatório de hi*ci para todo i de 1 a n_cli
    model.objective = xsum(demands[j]*y[j] for j in range(n_cli))

    #  Adicionando as restrições
        #  o somatório de lj para todo j de 1 a n_loc deve ser igual a n_fac
    model += xsum(x[i] for i in range(n_loc)) == n_fac
        #  para todo i, ci deve ser menor ou igual ao somatório de lj para todo j tal que dij <= rad
    for j in range(n_cli):
        model += y[j] <= xsum(x[i] for i in range(n_loc) if distances[i][j] <= rad)

    return model

In [ ]:
# Para o problema do exemplo:

locais = 4
facilidades = 2
clientes = 10
raio = 3

# Nesse caso, vamos considerar que a população de crianças entre os bairros é constante
# Ou seja, a demanda dos clientes é a mesma
demandas = [1 for i in range(clientes)]

distancias = [[ 7.,   6.,   2.8,  6.,   8.7,  5.4,  8.,  10.2,  7.2,  2.5],
              [ 7.2,  2.9,  8.5,  4.8,  0.4,  9.,   2.2,  1.8,  9.,   7.5],
              [ 2.,   4.9,  6.,   6.5,  7.,   9.,   6.8,  8.5,  2.,   2.2],
              [ 8.5,  5.5,  2.8,  1.5,  6.,   1.5,  4.5,  6.6,  9.5,  5.]]

model = create_MCP_model(locais, facilidades, clientes, raio, demandas, distancias)
solution = model.optimize()

print("Status = ", solution)
print("Valor da solução = ", model.objective_value)

print("\nLocais escolhidos:")
for v in model.vars:
    if v.name[0] == "x" and v.x > 0.00001:
        print(v.name)
print("\nClientes cobertos:")
for v in model.vars:
    if v.name[0] == "y" and v.x > 0.00001:
        print(v.name)

Status =  OptimizationStatus.OPTIMAL
Valor da solução =  7.0

Locais escolhidos:
x_2
x_3

Clientes cobertos:
y_1
y_2
y_5
y_7
y_8
y_9
y_10


Uma observação: Pode ter acontecido que as duas soluções encontradas nas duas vezes que resolvemos o problema sejam diferentes, ou que a solução mude se rodarmos o notebook novamente. Isso acontece pois ambas soluções são igualmente boas, já que a função objetivo em ambas assume o mesmo valor, e o solver não favorece uma pela outra. As duas soluções ótimas podem ser representadas graficamente das seguintes formas:

MC1.svg

MC1(1).drawio(1).svg